<a href="https://colab.research.google.com/github/iisuslik43/dul/blob/hw3/hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install pymorphy2 >> /dev/null
! pip install transformers >> /dev/null
! pip install wandb >> /dev/null
! wandb login b569130067842c47dc059479b75930c8f598012a

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [0]:
import numpy as np
import torch
import torch.utils.data
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm_notebook as tqdm
import pickle
import torch.nn.functional as F
import os
import csv

import wandb
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [0]:
def sample_data_1():
  count = 100000
  rand = np.random.RandomState(0)
  return [[1.0, 2.0]] + rand.randn(count, 2) * [[5.0, 1.0]]
def sample_data_2():
  count = 100000
  rand = np.random.RandomState(0)
  return [[1.0, 2.0]] + (rand.randn(count, 2) * [[5.0, 1.0]]).dot(
  [[np.sqrt(2) / 2, np.sqrt(2) / 2], [-np.sqrt(2) / 2, np.sqrt(2) / 2]])

def sample_data_3():
  count = 100000
  rand = np.random.RandomState(0)
  a = [[-1.5, 2.5]] + rand.randn(count // 3, 2) * 0.2
  b = [[1.5, 2.5]] + rand.randn(count // 3, 2) * 0.2
  c = np.c_[2 * np.cos(np.linspace(0, np.pi, count // 3)),
  -np.sin(np.linspace(0, np.pi, count // 3))]

  c += rand.randn(*c.shape) * 0.2
  data_x = np.concatenate([a, b, c], axis=0)
  data_y = np.array([0] * len(a) + [1] * len(b) + [2] * len(c))
  perm = rand.permutation(len(data_x))
  return data_x[perm], data_y[perm]

In [0]:
data1 = sample_data_1()
data2 = sample_data_2()
data3, y3 = sample_data_3()

In [0]:
class VAE1(nn.Module):
  def __init__(self, dim=2, latent_dim=2):
    super().__init__()
    self.dim = dim
    self.latent_dim = latent_dim

    self.normal = torch.distributions.Normal(torch.tensor([0.0]), torch.tensor([1.0]))

    self.encoder = nn.Sequential(
      nn.Linear(self.dim, 32),
      nn.ReLU(True)
    )

    self.mu_z = nn.Linear(32, self.latent_dim)
    self.gamma_z = nn.Linear(32, self.latent_dim ** 2)

        

    self.decoder = nn.Sequential(
      nn.Linear(2, 32),
      nn.ReLU(True)
    )

    self.mu_x = nn.Linear(32, self.dim)
    self.gamma_x = nn.Linear(32, self.dim ** 2)

  def sample(self, mu, gamma):
    if self.training:
        gamma = torch.exp(0.5 * gamma)
        #std_z = Variable(torch.from_numpy(np.random.normal(0, 1, size=sigma.size())).float())
        std_z = self.normal.sample(mu.shape).to(device).reshape(-1, 1, 2)
        encoding = (std_z @ gamma).reshape(-1, 2) + mu
        return encoding
    else:
        return mu

  def encode(self, x):
    x = self.encoder(x)
    mu = self.mu_z(x)
    gamma = self.gamma_z(x)
    gamma = gamma.reshape(-1, 2, 2)
    return mu, gamma

  def decode(self, z):
    z = self.decoder(z)
    mu = self.mu_x(z)
    gamma = self.gamma_x(z)
    gamma = gamma.reshape(-1, 2, 2)
    return mu, gamma

  def kl(self, mu, gamma):
    dim = mu.shape[1]
    return 0.5 * ((gamma * torch.eye(dim).to(device)).sum(dim=(1, 2)) + (mu**2).sum(dim=-1) - dim - torch.log(gamma.det())).sum() / mu.shape[0]

  def forward(self, x):
    mu_z, gamma_z = self.encode(x)
    z = self.sample(mu_z, gamma_z)
    mu_x, gamma_x = self.decode(z)
    x1 = self.sample(mu_x, gamma_x)
    return x1, self.kl(mu_z, gamma_z)
    

In [0]:
class VAELoss(nn.Module):
  def __init__(self):
    super().__init__()
    self.mse = nn.MSELoss()
  
  def forward(self, target, output, kl):
    l2 = self.mse(output, target)
    return l2 + kl

In [0]:

def train(model, criterion, data, n_epochs=3, use_wandb=False, batch_size=3):
  split = int(len(data) * 0.8)
  dataset_train = torch.utils.data.DataLoader(data[:split],
                                              batch_size=batch_size,
                                              shuffle=True)
  dataset_test = torch.utils.data.DataLoader(data[split:],
                                             batch_size=batch_size,
                                             shuffle=True)
  optimizer = torch.optim.Adam(params=model.parameters())
  if use_wandb:
    wandb.init(project="dul-hw3-vae")
    wandb.watch(model)
  for epoch in tqdm(range(1, n_epochs + 1)):
      for x_batch in tqdm(dataset_train):
          optimizer.zero_grad()
          x_batch = x_batch.float().to(device)
          output, kl = model(x_batch)
          loss = criterion(x_batch, output, kl)
          loss.backward()
          optimizer.step()

          train_loss = loss.item()
          kl_train = kl.item()

          with torch.no_grad():
              model.eval()
              for x_test in dataset_test:
                  x_test = x_test.float().to(device)
                  output_test, kl_test = model(x_test)
                  test_loss = criterion(x_test, output_test, kl_test).item()
                  kl_test = kl_test.item()
                  break
              model.train()
          if use_wandb:
            wandb.log({"Test Loss": test_loss,
                        "Train Loss": train_loss,
                      "KL test": kl_test,
                      "KL train": kl_train,
                      "L2 test": test_loss - kl_test,
                      "L2 train": train_loss - kl_train
                        })

In [0]:
model1 = VAE1()
model2 = VAE1()
model3 = VAE1()

In [114]:
train(model1.to(device), VAELoss(), data1, use_wandb=True)

In [0]:
train(model2.to(device), VAELoss(), data2, use_wandb=True)

In [0]:
train(model3.to(device), VAELoss(), data3, use_wandb=True)